In [1]:
#import necessary libraries
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Conv2D,MaxPooling2D
from keras.layers import LSTM, SimpleRNN

In [3]:
import numpy as np
a = open("data_shuffle_input.csv")
net_data_input = np.genfromtxt(a, delimiter=",")
b = open("data_shuffle_target.csv")
net_data_target = np.genfromtxt(b, delimiter=",")
c = open("mydata_shuffle_input.csv")
net_mydata_input = np.genfromtxt(c, delimiter=",")
d = open("mydata_shuffle_target.csv")
net_mydata_target = np.genfromtxt(d, delimiter=",")

In [4]:
import random
from sklearn.model_selection import cross_val_score

net_data_input = net_data_input.reshape(net_data_input.shape[0],28,28,1)
net_mydata_input = net_mydata_input.reshape(net_mydata_input.shape[0],28,28,1)
net_data_input = net_data_input / 255

In [5]:
acc_per_fold = []
loss_per_fold = []


In [6]:
num = []
for i in range(len(net_data_target)):
    if net_data_target[i][0] == 1:
        num.append(0)
    if net_data_target[i][1] == 1:
        num.append(1)
    if net_data_target[i][2] == 1:
        num.append(2)
    if net_data_target[i][3] == 1:
        num.append(3)
    if net_data_target[i][4] == 1:
        num.append(4)
    if net_data_target[i][5] == 1:
        num.append(5)
    if net_data_target[i][6] == 1:
        num.append(6)
    if net_data_target[i][7] == 1:
        num.append(7)
    if net_data_target[i][8] == 1:
        num.append(8)
    if net_data_target[i][9] == 1:
        num.append(9)
num = np.asarray(num)        
print(type(num))


<class 'numpy.ndarray'>


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5)
acc_per_fold = []
acc = []
loss = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(net_data_input, net_data_target, net_mydata_input):

  
  # Define the model architecture for net mydata
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='tanh', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
     # Generate a print
    # Fit data to model
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} mydata')
    history_mydata = model.fit(net_mydata_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    

    
    predictions_mydata_train = model.predict(net_mydata_input[train])
    classes_mydata_train = np.argmax(predictions_mydata_train, axis = 1)
    classes_mydata_train = tf.keras.utils.to_categorical(classes_mydata_train,10)

    predictions_mydata_test = model.predict(net_mydata_input[test])
    classes_mydata_test = np.argmax(predictions_mydata_test, axis = 1)
    classes_mydata_test = tf.keras.utils.to_categorical(classes_mydata_test,10)
    
    
    # Define the model architecture for net data
    rnn_model_data = Sequential()

    rnn_model_data.add(Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    rnn_model_data.add(MaxPooling2D((2, 2)))
    rnn_model_data.add(Flatten())
    rnn_model_data.add(Dense(100, activation='tanh', kernel_initializer='he_uniform'))
    rnn_model_data.add(Dense(10, activation='softmax'))
    # compile model
    rnn_model_data.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} data')

  # Fit data to model
    history_data = rnn_model_data.fit(net_data_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    predictions_data_train = rnn_model_data.predict(net_data_input[train])
    classes_data_train = np.argmax(predictions_data_train, axis = 1)
    classes_data_train = tf.keras.utils.to_categorical(classes_data_train,10)
    
    predictions_data_test = rnn_model_data.predict(net_data_input[test])
    classes_data_test = np.argmax(predictions_data_test, axis = 1)
    classes_data_test = tf.keras.utils.to_categorical(classes_data_test,10)
    
    #make train list input
    joined_list_train = []
    joined_list_test = []

    for i in range(800):
        b = [*classes_data_train[i], *classes_mydata_train[i]]
        joined_list_train.append(b)
        
    for j in range(200):
        a = [*classes_data_test[j], *classes_mydata_test[j]]
        joined_list_test.append(a)
    
    
    joined_list_train = np.asarray(joined_list_train)
    joined_list_test = np.asarray(joined_list_test)
    
    joined_list_train = joined_list_train.reshape(joined_list_train.shape[0],1,20)

    joined_list_test = joined_list_test.reshape(joined_list_test.shape[0],1,20)
    
  # Define the model architecture for net data
    print(joined_list_train[0].shape)

    rnn_model_outputs = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_outputs.add(LSTM(256,input_shape=joined_list_train[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_outputs.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_outputs.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} outputs')

  # Fit data to model
    history = rnn_model_outputs.fit(joined_list_train, net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    scores = rnn_model_outputs.evaluate(joined_list_test, net_data_target[test], verbose=0)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predictions = rnn_model_outputs.predict(joined_list_test)
    classes = np.argmax(predictions, axis = 1)
    print(num[test])
    acc_per_fold.append(accuracy_score(num[test],classes))
    
    print('accuracy is ' + str(accuracy_score(num[test],classes)))
    
  # Generate generalization metrics
    acc.append(history.history['acc'])
    loss.append(history.history['loss'])
    

  # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 346us/sample - loss: 1.7218 - acc: 0.4525
Epoch 2/50
800/800 [==============================] - 0s 235us/sample - loss: 0.7917 - acc: 0.7700
Epoch 3/50
800/800 [==============================] - 0s 229us/sample - loss: 0.5620 - acc: 0.8562
Epoch 4/50
800/800 [==============================] - 0s 255us/sample - loss: 0.4923 - acc: 0.8650
Epoch 5/50
800/800 [==============================] - 0s 242us/sample - loss: 0.4010 - acc: 0.8963
Epoch 6/50
800/800 [==============================] - 0s 238us/sample - loss: 0.3512 - acc: 0.9075
Epoch 7/50
800/800 [==============================] - 0s 231us/sample - loss: 0.3141 - acc: 0.9250
Epoch 8/50
800/800 [==============================] - 0s 234us/sample - loss: 0.2872 - acc: 0.9300
Epoch 9/50
800/800 [==============================] - 0s 240us/sample - loss: 0.2662 - 

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 1 data
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 327us/sample - loss: 1.4889 - acc: 0.4825
Epoch 2/50
800/800 [==============================] - 0s 230us/sample - loss: 0.4922 - acc: 0.8525
Epoch 3/50
800/800 [==============================] - 0s 240us/sample - loss: 0.3090 - acc: 0.9100
Epoch 4/50
800/800 [==============================] - 0s 241us/sample - loss: 0.2294 - acc: 0.9375
Epoch 5/50
800/800 [==============================] - 0s 239us/sample - loss: 0.1656 - acc: 0.9588
Epoch 6/50
800/800 [==============================] - 0s 244us/sample - loss: 0.1289 - acc: 0.9812
Epoch 7/50
800/800 [==============================] - 0s 237us/sample - loss: 0.1040 - acc: 0.9862
Epoch 8/50
800/800 [==============================] - 0s 230us/sample - loss: 0.0807 - acc: 0.9887
Epoch 9/50
800/800 [==============================] - 0s 242us/sample - loss: 0.0656 - ac

800/800 [==============================] - 0s 31us/sample - loss: 0.0272 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 0s 31us/sample - loss: 0.0252 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 0s 27us/sample - loss: 0.0234 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 0s 28us/sample - loss: 0.0218 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 0s 27us/sample - loss: 0.0204 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 0s 41us/sample - loss: 0.0192 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 0s 39us/sample - loss: 0.0180 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 0s 39us/sample - loss: 0.0169 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 0s 40us/sample - loss: 0.0160 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 0s 41us/sample - loss: 0.0151 - acc: 1.0000
Epoch 40/50
800/800 [=

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 5 data
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 327us/sample - loss: 1.4763 - acc: 0.5250
Epoch 2/50
800/800 [==============================] - 0s 238us/sample - loss: 0.4904 - acc: 0.8612
Epoch 3/50
800/800 [==============================] - 0s 241us/sample - loss: 0.3272 - acc: 0.9150
Epoch 4/50
800/800 [==============================] - 0s 240us/sample - loss: 0.2486 - acc: 0.9287
Epoch 5/50
800/800 [==============================] - 0s 233us/sample - loss: 0.1854 - acc: 0.9563
Epoch 6/50
800/800 [==============================] - 0s 244us/sample - loss: 0.1461 - acc: 0.9700
Epoch 7/50
800/800 [==============================] - 0s 243us/sample - loss: 0.1154 - acc: 0.9850
Epoch 8/50
800/800 [==============================] - 0s 237us/sample - loss: 0.0888 - acc: 0.9925
Epoch 9/50
800/800 [==============================] - 0s 243us/sample - loss: 0.0712 - ac

800/800 [==============================] - 0s 48us/sample - loss: 0.0255 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 0s 49us/sample - loss: 0.0235 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 0s 52us/sample - loss: 0.0218 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 0s 57us/sample - loss: 0.0203 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 0s 52us/sample - loss: 0.0189 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 0s 54us/sample - loss: 0.0177 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 0s 52us/sample - loss: 0.0166 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 0s 44us/sample - loss: 0.0156 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 0s 51us/sample - loss: 0.0147 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 0s 56us/sample - loss: 0.0138 - acc: 1.0000
Epoch 40/50
800/800 [=

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[6 6 6 1 8 0 6 9 9 2 0 3 6 7 1 3 3 5 8 0 5 7 2 3 9 0 5 7 2 5 2 3 8 5 8 0 8
 9 8 7 9 8 7 5 0 5 1 7 3 1 6 3 6 5 6 9 2 5 1 3 6 2 9 1 4 8 7 8 0 5 2 7 4 0
 2 9 0 2 2 7 5 8 8 2 2 9 6 9 4 2 0 0 5 7 3 9 1 8 5 1 1 9 0 4 2 0 1 4 0 1 2
 2 8 0 0 1 6 5 4 4 4 8 1 5 8 8 1 4 8 8 3 9 0 7 4 8 4 2 2 5 8 1 3 9 1 0 3 0
 8 3 4 9 2 4 0 6 8 0 2 9 9 5 7 1 7 2 3 5 7 5 1 8 5 0 1 2 1 5 1 4 8 4 0 2 9
 0 0 1 6 0 6 8 5 8 3 4 8 6 3 5]
accuracy is 0.885


In [8]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.38823790874332187 - Accuracy: 0.895%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.4582580402493477 - Accuracy: 0.865%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.31804237604141233 - Accuracy: 0.94%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.20972076952457427 - Accuracy: 0.93%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.3455034828186035 - Accuracy: 0.885%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 0.9030000000000001 (+- 0.02803569153775237)
> Loss: 0.3439525154754519
------------------------------------------------------------------------


In [9]:
np.savetxt('CNN_both_outputs_cross_validation_acc.csv',acc, delimiter=',')

In [10]:
np.savetxt('CNN_both_outputs_cross_validation_loss.csv',loss, delimiter=',')